In [1]:
from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset, Model, Run
from azureml.core.compute import ComputeTarget, AmlCompute, DataFactoryCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE, DEFAULT_GPU_IMAGE
from azureml.pipeline.core import Pipeline, PipelineParameter, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter, PipelineData
from azureml.data.output_dataset_config import OutputTabularDatasetConfig, OutputDatasetConfig, OutputFileDatasetConfig
from azureml.data.datapath import DataPath
from azureml.data.data_reference import DataReference
from azureml.data.sql_data_reference import SqlDataReference
from azureml.pipeline.steps import DataTransferStep
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [2]:
ws = Workspace.from_config()

In [3]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AksWebservice, LocalWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

model_name = 'Street_Segmentation_Model'

model_list = Model.list(ws, model_name)
best_child_run_id = model_list[0].run_id
best_child_run = Run.get(ws, best_child_run_id)
model_list[0].download(exist_ok=True)
model = model_list[0]
environment = Environment.from_conda_specification('instancesegmentationenv', './automl_outputs/conda_env.yml')
inference_config = InferenceConfig(entry_script='score.py', environment=environment, source_directory='./automl_outputs') # model_path = './automl_outputs/outputs/model.pt'

deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws,
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=deployment_config,
                           name='automl-image-test',
                           overwrite=True)
local_service.wait_for_deployment(show_output=False)
print(local_service.state)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 0a0e57407315484eb0d6cb59bfead822.azurecr.io
Logging into Docker registry 0a0e57407315484eb0d6cb59bfead822.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 0a0e57407315484eb0d6cb59bfead822.azurecr.io/azureml/azureml_b5850de23fb6599558e87c4b9e562b6e
 ---> 3f41b005ce0f
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> d0b24a678add
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImYzZTM4YWFhLWRkOWMtNGYxNy05NWMxLWVmM2ZmNDcyZGE2MSIsInJlc291cmNlR3JvdXBOYW1lIjoiY3ViaWNfaGFja2F0aG9uX3JnIiwiYWNjb3VudE5hbWUiOiJjdWJpYy1oYWNrLWFtbC13cyIsIndvcmtzcGFjZUlkIjoiMGEwZTU3NDAtNzMxNS00ODRlLWIwZDYtY2I1OWJmZWFkODIyIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 683f0d155b68
 ---> 860e42006e6a
Step 4/5 : RUN mv '/var/azureml-app/tmp7mdmhwno.py' /var/azureml-app/main.py
 ---> Runnin